# AnsProlog Basics
Unlike in Prolog, we don’t have to issue a query in order for computation to happen. The answer set solver instead produces all possible collections of facts that are consistent with what we’ve said in the program.

In [1]:
%%clingo -V0 0
% The answer set is calculated automatically without an explicit query.
p. % A fact.
q :- p. % q if p.

p q
SATISFIABLE


## Choices

In [2]:
%%clingo -V0 0

% Define conflicts
:- p, q.
:- r, s.
    
% Choose 2 to 3 elements from {p, q, r, s}.
2 {p; q; r; s} 3.

r p
r q
s p
s q
SATISFIABLE


## Graph Coloring

In [3]:
%%clingo -V0 0

% Define a graph with 6 nodes by counting all its edges.
node(1..6).
edge(1, 2; 1, 3; 1, 4).
edge(2, 4; 2, 6).
edge(3, 4).
edge(5, 4; 5, 6).

% There are 3 possible colors for a node.
color(r; g; b).

% For all C that satisfies color(C), each node can only has one C.
{hasColor(N, C) : color(C)} = 1 :- node(N).

% Negate all cases where adjacent nodes have the same color.
:- hasColor(N, C), hasColor(M, C), edge(N, M), color(C).

#show hasColor/2.

hasColor(2,r) hasColor(3,r) hasColor(5,r) hasColor(1,g) hasColor(6,g) hasColor(4,b)
hasColor(2,r) hasColor(3,r) hasColor(5,r) hasColor(1,g) hasColor(4,b) hasColor(6,b)
hasColor(2,r) hasColor(3,r) hasColor(1,g) hasColor(5,g) hasColor(4,b) hasColor(6,b)
hasColor(1,r) hasColor(2,g) hasColor(3,g) hasColor(5,g) hasColor(4,b) hasColor(6,b)
hasColor(1,r) hasColor(6,r) hasColor(2,g) hasColor(3,g) hasColor(5,g) hasColor(4,b)
hasColor(1,r) hasColor(5,r) hasColor(2,g) hasColor(3,g) hasColor(4,b) hasColor(6,b)
hasColor(1,r) hasColor(5,r) hasColor(4,g) hasColor(6,g) hasColor(2,b) hasColor(3,b)
hasColor(1,r) hasColor(6,r) hasColor(4,g) hasColor(2,b) hasColor(3,b) hasColor(5,b)
hasColor(1,r) hasColor(4,g) hasColor(6,g) hasColor(2,b) hasColor(3,b) hasColor(5,b)
hasColor(2,r) hasColor(3,r) hasColor(5,r) hasColor(4,g) hasColor(1,b) hasColor(6,b)
hasColor(2,r) hasColor(3,r) hasColor(4,g) hasColor(6,g) hasColor(1,b) hasColor(5,b)
hasColor(2,r) hasColor(3,r) hasColor(5,r) hasColor(4,g) hasColor(6,g) hasCol

## Tower of Hanoi

In [4]:
%%clingo -V0 0

% The number of levels of the tower.
level(2).

% The maximum round count. Should be 2^L-1.
maxRound(2 ** L - 1) :- level(L).
round(0..M) :- maxRound(M).

% Disks and stakes.
disk(D) :- level(L), D = 1..L.
stake(a;b;c).

% For a certain round and a certain disk, we can deduce the only stake.
{state(R, D, S) : stake(S)} = 1 :- round(R), disk(D).

% Beginning and end.
% In the round 0, for all disk D, D is on the stake 1.
state(0, D, a) :- disk(D).
state(M, D, b) :- disk(D), maxRound(M).

% What's moving?
moving(N, D, S, T) :- state(N + 1, D, T), state(N, D, S), S != T,
    % Must move the disk at the top.
    D = #min {K : disk(K), state(N, K, S)},
    % A larger disk cannot move onto a smaller one.
    D < #min {K : disk(K), state(N, K, T)}.
% Each round must contain a single move.
moving(N, D) :- moving(N, D, _, _).
:- {moving(N, D) : disk(D)} != 1, maxRound(M), N = 0..M-1.
% :- moving(N, D), disk(D), maxRound(M), N >= M.

% No moving backwards.
:- moving(N + 1, D, T, S), moving(N, D, S, T).

#show moving/4.

moving(0,1,a,c) moving(1,2,a,b) moving(2,1,c,b)
SATISFIABLE


In [5]:
%%clingo -V0 0

% Solution Given in the Example.

peg(a;b;c).
disk(1..4).
init_on(1..4,a).
goal_on(1..4,c).
moves(15).

% Generate
{ move(D,P,T) : disk(D), peg(P) } = 1 :- moves(M), T = 1..M.

% Define
move(D,T) :- move(D,_,T).

on(D,P,0) :- init_on(D,P).
on(D,P,T) :- move(D,P,T).
on(D,P,T+1) :- on(D,P,T), not move(D,T+1), not moves(T).

blocked(D-1,P,T+1) :- on(D,P,T), not moves(T).
blocked(D-1,P,T) :- blocked(D,P,T), disk(D).

% Test
:- move(D,P,T), blocked(D-1,P,T).
:- move(D,T), on(D,P,T-1), blocked(D,P,T).
:- goal_on(D,P), not on(D,P,M), moves(M).
:- { on(D,P,T) } != 1, disk(D), moves(M), T = 1..M.
    
% Display
#show move/3.

move(4,b,1) move(3,c,2) move(4,c,3) move(2,b,4) move(4,a,5) move(3,b,6) move(4,b,7) move(1,c,8) move(4,c,9) move(3,a,10) move(4,a,11) move(2,c,12) move(4,b,13) move(3,c,14) move(4,c,15)
SATISFIABLE
